In [2]:
!pip install Ipython --upgrade
%load_ext autoreload
%autoreload 2
%reload_ext autoreload

Requirement already up-to-date: Ipython in /usr/local/lib/python3.7/dist-packages (7.25.0)
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
from google.colab import drive, files
drive.mount('/content/drive')
rootpath = '/content/drive/Shareddrives/KW phase1 repo'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
import sys
sys.path.append(rootpath+'/package')
from utils.env import env

In [10]:
import cvxpy as cp
import csv

In [5]:
class SDP():
  def __init__(self):
    self._var_SDPt = None
    self._var_SOCPt = None
    self._var_p = None
    self._constraint = None


  def buildConstraint(self, targets, anchors, pathLossModel):
    soc_constraints = []

    for i, target_i in enumerate(targets):

      # Inner Constraint
      for j, target_j in enumerate(targets):
        if not target_i.id == target_j.id:
          rss = target_i.searchRssById(target_j.id)
          d = pathLossModel.estimateDistance(target_i.power, rss)

          pi = cp.reshape(self._var_p[i][-1][:3],(3,1))
          pj = cp.reshape(self._var_p[j][-1][:3],(3,1))
          # cp.SOC(t, x) means that the constraint: ||x||_2 <= t
          soc_constraints.append(cp.SOC([d] @ self._var_SOCPt, pi - pj))

      for anchor in anchors:
        rss = target_i.searchRssById(anchor.id)
        d = pathLossModel.estimateDistance(target_i.power, rss)
        xi = np.array([anchor.observedPosition]).T
        I = np.identity(3)
        firstRow = np.concatenate((I, (-xi)), axis=1)
        secondRow = np.concatenate((-(xi.T), xi.T @ xi), axis=1)
        Ai = np.concatenate((firstRow,secondRow), axis=0)
        soc_constraints.append(cp.trace(Ai @ self._var_p[i]) - cp.power(d,2) <= self._var_SDPt)
        soc_constraints.append(cp.trace(Ai @ self._var_p[i]) - cp.power(d,2) >= -self._var_SDPt)
        soc_constraints.append(self._var_p[i][-1][-1] == 1)

    self._constraint = soc_constraints

  def doSolve(self, log):
    prob = cp.Problem(cp.Minimize(self._var_SDPt+self._var_SOCPt), self._constraint)
    prob.solve(verbose=log)


  @property
  def var_SDPt(self):
    return self._var_SDPt

  @property
  def var_SOCPt(self):
    return self._var_SOCPt

  @property
  def var_p(self):
    return self._var_p

  @var_p.setter
  def var_p(self, p):
    self._var_p = p

  @var_SDPt.setter
  def var_SDPt(self, t):
    self._var_SDPt = t
  
  @var_SOCPt.setter
  def var_SOCPt(self, t):
    self._var_SOCPt = t

In [11]:
def calculateSDP(groupListSamples, UAVsSamples, savePath):
  sdpResultSamples = {}

  for sample, groupList in groupListSamples.items():
    print("#{}".format(sample), end='')
    sdp = SDP()
    sdp.var_SOCPt = cp.Variable(1)
    sdp.var_SDPt = cp.Variable(1)
    sdp.var_p = [cp.Variable((DIMENSION+1, DIMENSION+1), symmetric=True) for i in range(N_ngps)]

    targets = []
    for uavIndex, uav in UAVsSamples[sample].items():
      if uav.observedPosition is None:
        targets.append(uav)
    
    results = []
    for groupSize, groups in groupList.items():
      for group in groups:
        print(".", end='')
        anchors = []
        for i in group:
          anchors.append(UAVsSamples[sample][i])

        sdp.buildConstraint(targets, anchors, pathLossModel)
        sdp.doSolve(log=False)
        results.append([sdp.var_p[i].value[-1][:3] for i in range(N_ngps)])
    sdpResultSamples[sample] = results
    print("\n")

  sdpResultSamples_reform = {}
  for sample, socpResult in sdpResultSamples.items():
    socpResult_uav = []
    for i in range(np.array(socpResult).shape[1]):
      temp = []
      for j in range(np.array(socpResult).shape[0]):
        temp.append(socpResult[j][i])
      socpResult_uav.append(temp)
    sdpResultSamples_reform[sample] = socpResult_uav
  
  print(np.array(sdpResultSamples_reform[0]).shape)

  for i in range(len(dict1.keys())):
    for j in range(N_ngps):
      with open(savePath+"/position_{}.csv".format(j), "a") as _csv:
        csvWriter = csv.writer(_csv, delimiter=',')
        csvWriter.writerows(sdpResultSamples_reform[i][j])